In [ ]:
from shapely.geometry import Polygon, Point
import pandas as pd
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
%matplotlib inline

In [ ]:
zipcodes = gpd.read_file('../Data/zipcodes.geojson')

In [ ]:
zipcodes.plot();

In [ ]:
rta = gpd.read_file('../Data/Regional_Transportation_Authority_Bus_Stops.csv')

In [ ]:
scoot_trips = gpd.read_file('../Data/scooters_trips_clean.csv')

In [ ]:
scoot_trips.describe()

In [ ]:
scoot_trips.info()

In [ ]:
scoot_trips['geometry'] = scoot_trips.apply(lambda x: Point((float(x.startlongitude), float(x.startlatitude))), axis = 1)

scoot_trips.head()

In [ ]:
scoot_geo = gpd.GeoDataFrame(scoot_trips, 
                           crs = zipcodes.crs, 
                           geometry = scoot_trips['geometry'])

In [ ]:
scoot_by_zip = gpd.sjoin(scoot_geo, zipcodes, predicate = 'within')

In [ ]:
scoot_by_zip.head()

In [ ]:
scoot_zip_count = scoot_by_zip['zip'].value_counts()

In [ ]:
scoot_zip_count = scoot_zip_count.reset_index()

In [ ]:
scoot_zip_count.columns = ['zip','scoot_trips']

In [ ]:
bus_stops = pd.read_csv('../data/busstops_cleaned.csv')
print(bus_stops.shape)
bus_stops.head(3)

In [ ]:
bus_stops.apply(lambda x: Point((float(x.lng), float(x.lat))), axis = 1)

In [ ]:
bus_stops['geometry'] = bus_stops.apply(lambda x: Point((float(x.lng), 
                                                         float(x.lat))), 
                                        axis=1)
bus_stops.head(3)

In [ ]:
bus_geo = gpd.GeoDataFrame(bus_stops, 
                           crs = zipcodes.crs, 
                           geometry = bus_stops['geometry'])

In [ ]:
type(bus_geo)

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [ ]:
zipcodes

In [ ]:
stops_by_zip = gpd.sjoin(bus_geo, zipcodes, predicate = 'within')

In [ ]:
stops_by_zip.head()

In [ ]:
stop_zip_count = stops_by_zip['zip'].value_counts()
stop_zip_count = stop_zip_count.reset_index()

In [ ]:
stop_zip_count.columns = ['zip','bus_stops']

In [ ]:
zip_counts = pd.merge(scoot_zip_count, stop_zip_count, on = 'zip', how = 'outer')
zip_counts['scoot_trips per bus_stop'] = round(zip_counts.scoot_trips / zip_counts.bus_stops)
zip_counts

In [ ]:
type(zip_counts)

In [ ]:
zip2 = pd.merge(zipcodes, zip_counts, on = 'zip', how = 'outer')

In [ ]:
zip_counts.to_csv('../Data/zip_counts.csv')

In [ ]:
zip2.to_csv('../Data/zip2.csv')

In [ ]:
zip2.head()

In [ ]:
zip_counts.head()